# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import traceback
import gmaps
import os

# Import API key
from api_keys import g_key

In [48]:
### Store Part I results into DataFrame
# * Load the csv exported in Part I to a DataFrame

In [49]:
city_data_df = pd.read_csv("clean_city_data.csv", index_col=False)
city_data_df.reset_index(drop=True, inplace=True)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,avarua,-21.21,-159.78,84.20,66,40,16.11,CK,2020-05-17 01:13:32
1,rikitea,-23.12,-134.97,76.98,85,100,11.01,PF,2020-05-17 01:13:32
2,tiarei,-17.53,-149.33,87.80,70,40,11.41,PF,2020-05-17 01:13:32
3,albany,42.60,-73.97,62.76,49,54,2.48,US,2020-05-17 01:09:33
4,maningrida,-12.07,134.27,83.12,63,1,16.73,AU,2020-05-17 01:13:33
...,...,...,...,...,...,...,...,...,...
568,moses lake,47.13,-119.28,73.67,27,75,5.82,US,2020-05-17 01:15:56
569,anito,12.45,125.29,82.81,84,89,11.72,PH,2020-05-17 01:15:57
570,whitehorse,60.72,-135.05,48.54,70,75,17.22,CA,2020-05-17 01:13:57
571,hannibal,39.71,-91.36,67.08,88,90,4.70,US,2020-05-17 01:15:57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[['Lat', 'Lng']].astype(float)

# Convert Humidity to float and store
humidity = city_data_df['Humidity'].astype(float)
print(len(locations))
print(len(humidity))

573
573


In [51]:
# Create a poverty Heatmap layer
starting_coordinates = (38.58, -121.48) #Sac starting point
fig = gmaps.figure(center=starting_coordinates, 
                   zoom_level=4,
                   map_type='HYBRID')

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False,         
                                 max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:
temp_df = city_data_df[(city_data_df['Max Temp'] < 90)  & (city_data_df['Max Temp'] > 60)]
temp_df = temp_df[(temp_df['Humidity'] < 80)  & (temp_df['Humidity'] > 60)]
temp_df = temp_df[(temp_df['Wind Speed'] < 40)  & (temp_df['Wind Speed'] > 10)]
temp_df = temp_df.dropna()
temp_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,avarua,-21.21,-159.78,84.20,66,40,16.11,CK,2020-05-17 01:13:32
2,tiarei,-17.53,-149.33,87.80,70,40,11.41,PF,2020-05-17 01:13:32
4,maningrida,-12.07,134.27,83.12,63,1,16.73,AU,2020-05-17 01:13:33
9,atuona,-9.80,-139.03,81.18,78,20,17.43,PF,2020-05-17 01:13:34
15,santa marta,11.24,-74.20,84.00,78,100,12.35,CO,2020-05-17 01:13:35
19,iralaya,15.00,-83.23,86.00,74,100,11.41,HN,2020-05-17 01:13:36
27,san patricio,19.22,-104.70,80.60,78,5,11.41,MX,2020-05-17 01:13:39
31,isangel,-19.55,169.27,77.00,69,73,11.41,VU,2020-05-17 01:13:40
39,cidreira,-30.18,-50.21,61.83,70,0,11.72,BR,2020-05-17 01:13:41
45,puro,12.48,123.38,85.46,73,100,10.58,PH,2020-05-17 01:13:43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [53]:
lat = []
lng = []
hotels = []
cities = []
countries = []
vicinities = []
ratings = []

for index, row in temp_df.iterrows():
    print("--------")
    print(row['City'])

    # geocoordinates
    target_coordinates = (f"{row['Lat']}, {row['Lng']}")
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    target_rank = "rating"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "rankBy": target_rank,
        "key": g_key
    }

    try:
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        #print(response.url)
        # convert response to json
        results = response.json()
        #print(response.json())
        # Print the json (pretty printed)
        #print(json.dumps(results, indent=4, sort_keys=True))

        hotels.append(results["results"][0]["name"])
        lat.append(row['Lat'])
        lng.append(row['Lng'])
        cities.append(row['City'])
        countries.append(row['Country'])
        vicinities.append(results["results"][0]["vicinity"])
        ratings.append(results["results"][0]["rating"])

        print(results["results"][0]["rating"])
        print(results["results"][0]["name"])
        print(results["results"][0]["vicinity"])
        
    except Exception as e:
        print(f"###{e}###")
        print(f"### No hotel(S) found for city: {row['City']}. Skipping...")
        print(traceback.format_exc())

--------
avarua
4.1
The Edgewater Resort and Spa
Arorangi District
--------
tiarei
4.3
Le Rocher de Tahiti
Faananu
--------
maningrida
4.3
Dhukurrdji Lodge
Maningrida
--------
atuona
0
Simplicité Marquises
Hiva Oa
--------
santa marta
4.5
Masaya Hostel Santa Marta
Distrito Turístico Cultural E Histórico, Cl. 14 ##4 - 80
--------
iralaya
###list index out of range###
### No hotel(S) found for city: iralaya. Skipping...
Traceback (most recent call last):
  File "<ipython-input-53-4bff4fa02196>", line 43, in <module>
    hotels.append(results["results"][0]["name"])
IndexError: list index out of range

--------
san patricio
4.5
Hotel Hacienda de Melaque
Morelos 49, Centro, San Patricio
--------
isangel
0
Quarumanu area
Loanengo village, Tanna Island
--------
cidreira
4.6
Pond Country Club
Estrada RS 784 KM 7 - Lagoa Country Club, Cidreira
--------
puro
###list index out of range###
### No hotel(S) found for city: puro. Skipping...
Traceback (most recent call last):
  File "<ipython-input-5

In [54]:
hotel_df = pd.DataFrame({'Hotel Name':hotels,
                        'City':cities,
                        'Country':countries,
                        'Lat':lat,
                        'Lng':lng,
                        'Vicinity':vicinities,
                        'Rating':ratings})
hotel_df
#hotel_df.sort_values(['Country'], ascending=True)

,Hotel Name,City,Country,Lat,Lng,Vicinity,Rating
0,The Edgewater Resort and Spa,avarua,CK,-21.21,-159.78,Arorangi District,4.1
1,Le Rocher de Tahiti,tiarei,PF,-17.53,-149.33,Faananu,4.3
2,Dhukurrdji Lodge,maningrida,AU,-12.07,134.27,Maningrida,4.3
3,Simplicité Marquises,atuona,PF,-9.80,-139.03,Hiva Oa,0.0
4,Masaya Hostel Santa Marta,santa marta,CO,11.24,-74.20,"Distrito Turístico Cultural E Histórico, Cl. 1...",4.5
5,Hotel Hacienda de Melaque,san patricio,MX,19.22,-104.70,"Morelos 49, Centro, San Patricio",4.5
6,Quarumanu area,isangel,VU,-19.55,169.27,"Loanengo village, Tanna Island",0.0
7,Pond Country Club,cidreira,BR,-30.18,-50.21,"Estrada RS 784 KM 7 - Lagoa Country Club, Cidr...",4.6
8,梨树韩州宾馆,lishu,CN,43.30,124.33,梨树前进大街与经开大路交汇公安局旁,0.0
9,Sheraton Kauai Resort at Coconut Beach,kapaa,US,22.08,-159.32,"650 Aleka Loop, Kapaʻa",4.1


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Add marker layer ontop of heat map
# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# display the combined figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))